# Necessary Imports

In [1]:
import os
import numpy as np
import pandas as pd
import shutil
import warnings
warnings.filterwarnings("ignore")

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


# Data Processig and Feature Extraction

In [2]:
class Data_handling:

    def __init__(self,file_path):
        self.df = pd.read_csv(file_path)

    def split_dataset_to_X_Y_Z(self,save_path,file_type):
        df_name = pd.DataFrame(columns=range(21))
        for col in ['X','Y','Z']:
            k=0
            for i in range(0,len(self.df[col]),20):
                if(len(self.df[col][i:i+20].values)==20): 
                    df_name.loc[k] = np.append(self.df[col][i:i+20].values, self.df['y'][i])
                    k+=1
            df_name.to_csv(save_path+file_type+'_'+col+'.csv',index=False)
        return

    @staticmethod
    def merge_csv():
        for col in ['Accelerometer.csv','Gyroscope.csv']:
            df_climb = pd.read_csv('Data/new/climbing_0/'+col)
            df_jump = pd.read_csv('Data/new/jumping_1/'+col)
            df_run = pd.read_csv('Data/new/running_2/'+col)
            df_walk = pd.read_csv('Data/new/walking_3/'+col)
            df = df_climb.append(df_jump.append(df_run.append(df_walk,ignore_index = True),ignore_index = True),ignore_index = True)
            df.to_csv('Data/new/'+col,index=False)
        return

# Model Building and Training

In [3]:
class Model:


    def __init__(self,file_path):
        (X,Y) = self.get_data(file_path)
        self.kerras_classifier(X,Y)
        return
        

    def get_data(self,file_path):
        df = pd.read_csv(file_path)

        dataset = df.values

        X = dataset[:,0:20].astype(float) # sensor data
        Y = dataset[:,20].astype(int) # labels
        return (X,Y)

    def create_model(self):
        # Define model
        model = Sequential()
        model.add(Dense(22, input_dim=20, activation='relu'))
        model.add(Dense(22, activation='relu'))
        model.add(Dense(22, activation='relu'))
        model.add(Dense(4, activation='softmax'))

        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    def kerras_classifier(self,X_train,Y_train):
        self.model = KerasClassifier(self.create_model, epochs=200, batch_size=100, verbose=False)
        self.model.fit(X_train, Y_train)
        return self.model

    def result(self,X_test,Y_test):
        kfold = KFold(n_splits=10, shuffle=True, random_state=5)
        cv_results = cross_val_score(self.model, X_test, Y_test, cv=kfold)
        print("Baseline on test data: %.2f%% (%.2f%%)" % (cv_results.mean()*100, cv_results.std()*100))
        return cv_results

    def predict(self,X):
        return self.model.predict(X)

In [4]:
class activity_classifier:

    def __init__(self):
        print('training model 1'+'-'*50)
        self.acc_x_model = Model('Data/new/acc_X.csv')
        print('training model 2'+'-'*50)
        self.acc_y_model = Model('Data/new/acc_Y.csv')
        print('training model 3'+'-'*50)
        self.acc_z_model = Model('Data/new/acc_Z.csv')
        print('training model 4'+'-'*50)
        self.gyro_x_model = Model('Data/new/gyro_X.csv')
        print('training model 5'+'-'*50)
        self.gyro_y_model = Model('Data/new/gyro_Y.csv')
        print('training model 6'+'-'*50)
        self.gyro_z_model = Model('Data/new/gyro_Z.csv')
        return

    def accuracy(self):
        print('Calculating Accuracy'+'-'*50)
        df_acc = pd.read_csv('Data/new/Accelerometer.csv')
        df_gyro = pd.read_csv('Data/new/Gyroscope.csv')
        count=0
        total=0
        confussion = [
            [0,0,0,0],
            [0,0,0,0],
            [0,0,0,0],
            [0,0,0,0]
        ]
        
        for i in range(0,len(df_acc['X']),20):
            if(len(df_acc['X'][i:i+20].values)==20):
                p_acc_x = self.acc_x_model.predict(np.asarray([df_acc['X'][i:i+20].values]))
                p_acc_y = self.acc_y_model.predict(np.asarray([df_acc['Y'][i:i+20].values]))
                p_acc_z = self.acc_z_model.predict(np.asarray([df_acc['Z'][i:i+20].values]))
                p_gyro_x = self.gyro_x_model.predict(np.asarray([df_gyro['X'][i:i+20].values]))
                p_gyro_y = self.gyro_y_model.predict(np.asarray([df_gyro['Y'][i:i+20].values]))
                p_gyro_z = self.gyro_z_model.predict(np.asarray([df_gyro['Z'][i:i+20].values]))
                prediction = self.mode([p_acc_x[0],p_acc_y[0],p_acc_z[0],p_gyro_x[0],p_gyro_y[0],p_gyro_z[0]])
                if(df_acc['y'][i] == prediction):
                    count+=1
                total+=1
                confussion[df_acc['y'][i]][prediction] += 1
        return ((count/total)*100,confussion)

    def mode(self,array):
        return max(set(array), key = array.count) 
        # np.bincount(array).argmax()

In [5]:
models = activity_classifier()
result = models.accuracy()
print()
print(' '+'-'*79)
print('|' + ' '*25 +'Accuracy of the model:  '+str(result[0])+ ' '*25+'|')
print(' '+'-'*79)

training model 1--------------------------------------------------

training model 2--------------------------------------------------
training model 3--------------------------------------------------
training model 4--------------------------------------------------
training model 5--------------------------------------------------
training model 6--------------------------------------------------
Calculating Accuracy--------------------------------------------------

 -------------------------------------------------------------------------------
|                         Accuracy of the model:  96.12359550561797                         |
 -------------------------------------------------------------------------------


In [10]:
for i in range(4):
    for j in range(4):
        print(str(result[1][i][j]),end="  ")
    print()

412  2  0  16  
5  351  1  13  
0  5  429  1  
22  4  0  519  
